In [1]:
import pandas as pd
import os
import numpy as np

In [3]:
data_folder="./Data/current_FAO/raw_files/"
for file in os.listdir(data_folder):
    file=data_folder+file
    print("\n"+file)
    try:
        print(pd.read_csv(file,nrows=2).head())
    except:
       pass 



./Data/current_FAO/raw_files/ASTI_Research_Spending_E_All_Data_(Norm).csv
   Country Code  Country  Item Code                           Item  \
0             4  Algeria      23045  Agriculture research spending   
1             4  Algeria      23045  Agriculture research spending   

   Element Code                                            Element  Year Code  \
0          6083  Share of Value Added (Agriculture, Forestry an...       2009   
1          6083  Share of Value Added (Agriculture, Forestry an...       2010   

   Year  Unit  Value Flag  
0  2009   NaN   0.18   AS  
1  2010   NaN   0.18   AS  

./Data/current_FAO/raw_files/ASTI_Researchers_E_All_Data_(Norm).csv
   Country Code  Country  Item Code                            Item  \
0             4  Algeria      23046  Agricultural researchers (FTE)   
1             4  Algeria      23046  Agricultural researchers (FTE)   

   Element Code Element  Year Code  Year Unit  Value Flag  
0          6085  Number       2009  2009   

In [2]:
crops_normalised_filename="./Data/current_FAO/raw_files/Production_Crops_E_All_Data_(Normalized).csv"
crop_filename="./Data/fao_data_crops_data.csv"

crop_normalised_df=pd.read_csv(crops_normalised_filename,encoding = "ISO-8859-1")
crop_df=pd.read_csv(crop_filename)

In [38]:
crop_normalised_df.head(2)

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,1975,ha,0.0,F
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,1976,ha,5900.0,F


In [29]:
crop_normalised_df[['Element','Value']].groupby('Element').sum().sort_values('Value',ascending=False)

,Value
Element,
Production,2.996054e+12
Area harvested,9.142499e+11
Seed,9.317855e+10
Yield,6.928174e+10


In [30]:
crop_normalised_df[['Area','Element','Value']].groupby(['Area','Element']).count()

Value
Area        Element              
Afghanistan Area harvested   2470
            Production       2578
            Seed              972
            Yield            2467
Africa      Area harvested   8219
...                           ...
Zambia      Yield            2011
Zimbabwe    Area harvested   3438
            Production       3799
            Seed             1134
            Yield            3417

[1003 rows x 1 columns]

In [3]:
crop_types_df=crop_normalised_df[['Item','Value']].groupby('Item').sum().sort_values('Value',ascending=False)

In [4]:
crop_types_df.sample(5)

,Value
Item,
Cucumbers and gherkins,9.207374e+09
Blueberries,9.780348e+07
Buckwheat,1.709648e+09
Vetches,6.994790e+08
"Fruit, pome nes",1.118760e+07


In [13]:
files=[\
      "./Data/current_FAO/raw_files/CommodityBalances_Crops_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Environment_LandCover_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Environment_LivestockPatterns_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_Crops_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_CropsProcessed_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_Indices_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_Livestock_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_LivestockPrimary_E_All_Data_(Normalized).csv",\
      "./Data/current_FAO/raw_files/Production_LivestockProcessed_E_All_Data_(Normalized).csv"]

In [5]:
def store_panda_df(url, year, id_conf ):
    
    result = requests.get(url)
    
    soupe = BeautifulSoup(result.text, 'html.parser')
    publications_list = []
    elements = soupe.body.find_all('ul')[1].find_all('li')
    for element in elements:
        
        title = element.find_all('a', href = re.compile("^/paper"))[0].text
        authors_list = element.find_all('a',{"class" : 'author'} )
        
        authors = [author.text for author in authors_list]
        
        publications_list.append({'id': id_conf, 'authors': authors, 'year': year, 'title': title})
        
    publications = pd.DataFrame.from_dict(publications_list)
    
    return publications

In [6]:
from requests import get
import requests
from bs4 import BeautifulSoup
import re

def get_category(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.text,'html.parser')
    categories=soup.body.find_all('a')
    

In [7]:
url="https://world-crops.com/food-crops/"
r=requests.get(url,headers={"User-Agent": "XY"})
soup=BeautifulSoup(r.text,'html.parser')
elements_temp=soup.find_all('a',href=re.compile("^../"))
elements=[el.text for el in elements_temp]

for i in range(40,401,40):
    url_i=url+"?ss="+str(i)
    r=requests.get(url_i,headers={"User-Agent":"XY"})
    soup=BeautifulSoup(r.text,'html.parser')
    new_elements=soup.find_all('a',href=re.compile("^../"))
    elements+=[el.text for el in new_elements]


In [8]:
elements_stable=elements

TypeError: descriptor 'lower' of 'str' object needs an argument

In [9]:
crop_types_df.index=crop_types_df.index.str.lower()
elements=list(map(lambda x:x.lower(),elements))


In [69]:
def inclusive_search(string,elements):
    
    delimiters = ",", "(","&",")"," and "," "
    pattern = '|'.join(map(re.escape, delimiters))
    strings=list(filter(None,re.split(pattern,string)))
    found=False
    for s in strings:
        if s=="nes":
            continue
        for el in elements:
            found=(s in el.split())
            if found==False and s[-1]=="s":
                found=s[:-1] in el.split()
            if found==False and s[-2:]=="es":
                found=s[:-2] in el.split()
            if found==False and s[-3:]=="ies":
                found=s[:-3]+"y" in el.split()
            if found==True:
                return found
    return found

In [74]:
test=crop_types_df[list(map(lambda x:not inclusive_search(x,elements),crop_types_df.reset_index().Item))]
test

,Value
Item,
"cereals,total",6.476818e+11
"coarse grain, total",2.923431e+11
vegetables primary,1.502298e+11
oilcakes equivalent,8.456278e+10
oilcrops primary,6.878902e+10
"vegetables, fresh nes",4.365071e+10
"pulses,total",3.450090e+10
seed cotton,2.217389e+10
fibre crops primary,1.607477e+10


In [66]:
string="coarse and buckle"
"coarse" in string.split()

True

In [249]:
elements

['abiu',
 'acerola',
 'achacha',
 'ackee',
 'african eggplant',
 'african mangosteen',
 'african rice',
 'agati',
 'air potato',
 'alpine currant',
 'amaranthus spp',
 'ambarella',
 'american gooseberry',
 'american groundnut',
 'american oil palm',
 'american persimmon',
 'apple',
 'apricot',
 'arazá',
 'armenian cucumber',
 'arracacha',
 'arrowleaf elephant ear',
 'arrowroot',
 'artichoke',
 'ash gourd',
 'asian palmyra palm',
 'asian pear',
 'asparagus',
 'atemoya',
 'australian desert raisin',
 'avocado',
 'azarole',
 'azuki bean',
 'babaco',
 'bael',
 'bambara groundnut',
 'bamboo',
 'banana',
 'barbados gooseberry',
 'barley',
 'beet',
 'beetroot',
 'bergamot',
 'betel nut',
 'bignay',
 'bilberry',
 'bilimbi',
 'binjai',
 'biriba',
 'bitter gourd',
 'bitter orange',
 'bitter vetch',
 'bitterleaf',
 'black chokeberry',
 'black currant',
 'black mulberry',
 'black mustard',
 'black radish',
 'black salsify',
 'black sapote',
 'blackberry',
 'blanched celery',
 'blue-berried honeysu